In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
ROOT_DIR = '/content/gdrive/MyDrive/Project'

In [4]:
!ls /content/gdrive/MyDrive/

'00_VeraCrypt Containers'   Documents	     Project
'Admit Card '		   'Dot Files'	     Report_Edited.docx
'Ankita Patra'		    DotFiles	    'Results '
'Avipsa Patra'		   'MLBB '	    'Semester - 5'
'Avisek Patra'		   'Number Plate '  'Semester 6'
'BFL Receipt'		    Obsidian	    'Untitled document.gdoc'


In [5]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [6]:
!pip install imutils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
!pip install opencv-contrib-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
!pip install ultralytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.1/510.1 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 18.0 MB/s eta 0:00:00


In [9]:
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
!pip install deskew

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import functools
import math
from typing import Union, Tuple

import cv2
import numpy as np
from deskew import determine_skew
from imutils import paths
import tensorflow
from tensorflow import keras
from tensorflow.keras.utils import img_to_array
from ultralytics import YOLO
from google.colab.patches import cv2_imshow


# Function from deskew to calculate rotation angle
def rotate(
        image: np.ndarray, angle: float, background: Union[int, Tuple[int, int, int]]
) -> np.ndarray:
    old_width, old_height = image.shape[:2]
    angle_radian = math.radians(angle)
    width_ = abs(np.sin(angle_radian) * old_height) + abs(np.cos(angle_radian) * old_width)
    height_ = abs(np.sin(angle_radian) * old_width) + abs(np.cos(angle_radian) * old_height)

    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    rot_mat[1, 2] += (width_ - old_width) / 2
    rot_mat[0, 2] += (height_ - old_height) / 2
    return cv2.warpAffine(image, rot_mat, (int(round(height_)), int(round(width_))), borderValue=background)


# Function to show output
def show_img(image):
    cv2_imshow(image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()


def get_license_plate(stock_image):
    # Load the YOLOv8 Model
    model = YOLO('/content/gdrive/MyDrive/Project/weights/best.pt')

    # Apply YOLOv8 model predictions and show the license plate
    results = model(stock_image)

    plotted = results[0].plot(show_conf=True)
    show_img(plotted)

    # Get the bounding box from YOLOv8 Model Prediction results
    box = results[0].boxes
    bbox = box.xyxy
    bbox = bbox.numpy()

    # Store the XY Coordinates in integer format
    x1 = int(bbox[0, 0])
    y1 = int(bbox[0, 1])
    x2 = int(bbox[0, 2])
    y2 = int(bbox[0, 3])

    # Now crop the license plate in the stock image
    cropped = stock_image[y1:y2, x1:x2]
    return cropped


def shadow_remove(img):
    rgb_planes = cv2.split(img)
    result_norm_planes = []
    for plane in rgb_planes:
        dilated_img = cv2.dilate(plane, np.ones((13, 13), np.uint8))
        bg_img = cv2.medianBlur(dilated_img, 35)
        diff_img = 255 - cv2.absdiff(plane, bg_img)
        norm_img = cv2.normalize(diff_img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
        result_norm_planes.append(norm_img)
    shadowremov = cv2.merge(result_norm_planes)
    return shadowremov


def preprocess_lPlate(image):
    # Show the Cropped License Plate
    show_img(image)

    # Deskew
    angle = determine_skew(image)
    rotated = rotate(image, angle, (0, 0, 0))
    show_img(rotated)

    # Remove shadows for cleaning license plate
    clear = shadow_remove(rotated)
    show_img(clear)

    # Convert to gray scale
    gray = cv2.cvtColor(clear, cv2.COLOR_BGR2GRAY)
    show_img( gray)

    # Denoise
    den = cv2.bilateralFilter(gray, 21, 75, 75)
    show_img(den)

    # Threshold using Otsu's Thresholding
    thresh = cv2.threshold(den, 0, 200, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    show_img( thresh)

    # Erosion & Dilation
    img_erode = cv2.erode(thresh, (3, 3))
    img_dilate = cv2.dilate(img_erode, (3, 3))
    show_img(img_dilate)

    clean_license_plate(img_dilate, gray.shape)
    return img_dilate


def clean_license_plate(lic_plate, shape):
    # apply connected component analysis to the threshold image
    output = cv2.connectedComponentsWithStats(lic_plate, 4, cv2.CV_32S)
    (numLabels, labels, stats, centroids) = output
    # initialize an output mask to store all characters parsed from
    # the license plate
    mask = np.zeros(shape, dtype="uint8")
    # loop over the number of unique connected component labels, skipping
    # over the first label (as label zero is the background)
    for i in range(1, numLabels):
        # extract the connected component statistics for the current
        # label
        x = stats[i, cv2.CC_STAT_LEFT]
        y = stats[i, cv2.CC_STAT_TOP]
        w = stats[i, cv2.CC_STAT_WIDTH]
        h = stats[i, cv2.CC_STAT_HEIGHT]
        area = stats[i, cv2.CC_STAT_AREA]

        # print(w, h, area)

        # ensure the width, height, and area are all neither too small
        # nor too big
        keepWidth = 20 < w < 60
        keepHeight = 75 < h < 115
        keepArea = 1250 < area < 4000
        # ensure the connected component we are examining passes all
        # three tests
        if all((keepWidth, keepHeight, keepArea)):
            # construct a mask for the current connected component and
            # then take the bitwise OR with the mask
            componentMask = (labels == i).astype("uint8") * 255
            mask = cv2.bitwise_or(mask, componentMask)
            # show_img("mask", mask)

    show_img(mask)
    segment_lic_plate(mask)
    return mask


def segment_lic_plate(img):
    # Find contours and get bounding box for each contour
    contours, _ = cv2.findContours(img.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    boundingBoxes = [cv2.boundingRect(c) for c in contours]

    # Sort the bounding boxes from left to right, top to bottom
    # sort by Y first, and then sort by X if Ys are similar
    def compare(rect1, rect2):
        if abs(rect1[1] - rect2[1]) > 10:
            return rect1[1] - rect2[1]
        else:
            return rect1[0] - rect2[0]

    boundingBoxes = sorted(boundingBoxes, key=functools.cmp_to_key(compare))

    # Draw bounding boxes
    new = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    for i in range(len(boundingBoxes)):
        x, y, w, h = boundingBoxes[i]
        cv2.rectangle(new, (x, y), (x + w, y + h), (0, 255, 0), 2)
    show_img( new)
    apply_ocr(img, boundingBoxes)
    return new


def apply_ocr(image, boundingBoxes):
    color_img = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    # Define constants
    TARGET_WIDTH = 128
    TARGET_HEIGHT = 128

    chars = [
        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G',
        'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'
    ]

    # Load the pre-trained convolutional neural network
    load = tensorflow.keras.models.load_model('/content/gdrive/MyDrive/Project/weights/characters_model.weights')
    vehicle_plate = ""
    # Loop over the bounding boxes
    for rect in boundingBoxes:
        # Get the coordinates from the bounding box
        x, y, w, h = rect

        # Crop the character from the mask
        # and apply bitwise_not because in our training data for pre-trained model
        # the characters are black on a white background
        crop = image[y:y + h, x:x + w]
        crop = cv2.bitwise_not(crop)

        # Get the number of rows and columns for each cropped image
        # and calculate the padding to match the image input of pre-trained model
        rows = crop.shape[0]
        columns = crop.shape[1]
        paddingY = (TARGET_HEIGHT - rows) // 2 if rows < TARGET_HEIGHT else int(0.17 * rows)
        paddingX = (TARGET_WIDTH - columns) // 2 if columns < TARGET_WIDTH else int(0.45 * columns)

        # Apply padding to make the image fit for neural network model
        crop = cv2.copyMakeBorder(crop, paddingY, paddingY, paddingX, paddingX, cv2.BORDER_CONSTANT, None, 255)

        # Convert and resize image
        crop = cv2.cvtColor(crop, cv2.COLOR_GRAY2RGB)
        crop = cv2.resize(crop, (TARGET_WIDTH, TARGET_HEIGHT))
        # show_img("Cropped Characters", crop)

        # Prepare data for prediction
        crop = crop.astype("float") / 255.0
        crop = img_to_array(crop)
        crop = np.expand_dims(crop, axis=0)

        # Make prediction
        prob = load.predict(crop)[0]
        idx = np.argsort(prob)[-1]
        vehicle_plate += chars[idx]

        # Show bounding box and prediction on image
        cv2.rectangle(color_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(color_img, chars[idx], (x - 15, y), font, 0.8, (0, 0, 255), 2)

    show_img(color_img)
    print("Vehicle plate: " + vehicle_plate)
    return vehicle_plate


# This is for mass input
IMG_PATH = '/content/gdrive/MyDrive/Project/bike_samples/'
filenames = sorted(list(paths.list_images(IMG_PATH)), reverse=False)
print(filenames)

# Get the license plates using Y0L0v8 Model in the Predefined Resolution
for file in filenames:
    org_img = cv2.imread(file)
    img_cvt = cv2.resize(org_img, (1280, 720))
    # show_img(img_cvt)

    # Detect License plate using YOLOv8 Model
    license_plate = get_license_plate(img_cvt)

    # Resize for standardized size
    height, width, channels = license_plate.shape
    normal_format = license_plate
    if height < 150 and width < 350:
        normal_format = cv2.resize(license_plate, (500, 250), interpolation=cv2.INTER_CUBIC)

    # Start ANPR
    lPlate = preprocess_lPlate(normal_format)
